# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices. **This Series should be sorted by a tie-break sort in the format of ("extracted date", "original row number").**

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [8]:
import pandas as pd

pd.set_option('display.max_rows', None)

doc = []
with open('assets/dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [73]:
def date_sorter(df):

    # Each line is for matching with one of the designated types in the bulletpoints above.
    date_patterns = (
        r"\d{1,2}(?:/|-)\d{1,2}(?:/|-)(?:\d{2}|\d{4})\b|"
        r"(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)+\w*\.*(?:-|\s)+\d{1,2},*(?:-|\s)\d{4}|"
        r"\d{1,2}\s(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)+\w*\.*,*\s\d{2,4}|"
        r"(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)+\w*\.*\s\d{1,2}\w{,2},*\s\d{2,4}"
    )
    # These two patterns require some manipulation to add missing days and months.
    missing_day_1 = r"(?P<month>\d{1,2})/(?P<year>\d{4})" # We use named capturing groups to later pass on to .replace() method.
    missing_day_2 = r"(?P<month>jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)+\w*\.*,*\s(?P<year>\d{4})"
    # missing_month_day = r"\w*(?P<year>[12][09][0-9]{2})"
    year_only = r"\w*(?P<year>[12][09][0-9]{2})"

    # Turn all strings to lower case.
    lower = df.str.lower()
    
    # Filtering missing_days as a separate Series with the original index.
    missing_days = lower[~ lower.str.contains(date_patterns, regex=True)]

    # Replacing missing days with frist days of month.
    fixed_days = missing_days.str.replace(missing_day_1, r"\g<month>/01/\g<year>", regex=True) # We use the named capturing groups created above.
    fixed_days = fixed_days.str.replace(missing_day_2, r"1 \g<month> \g<year>", regex=True)
    
    # Filtering the data again to isolate the rows that only have years (no months or days).
    year_onlys = fixed_days[~ fixed_days.str.contains(date_patterns, regex=True)]
    
    # Replacing missing days and months with day 1 and month 1
    fixed_years = year_onlys.str.replace(year_only, r"01/01/\g<year>", regex=True)

    # Updating the original Series with the corrected dates.
    fixed_days.update(fixed_years)
    lower.update(fixed_days)
    
    # Returning all the matching date_patterns.
    dates = lower.str.findall(date_patterns)

    # Converting all the row values to datetime objects (iterating because of inconsistent formats.)
    for i, date in enumerate(dates):
        converted = pd.to_datetime(date)
        # Some values incorretly get mapped to future dates by offset of 100 years. We fix those.
        if converted.year > 2024:
            converted = converted - pd.DateOffset(years=100)
        dates[i] = converted[0]

    # Converting the series to dataframe to implement sort by two keys.
    order = dates.reset_index()
    order.index.name = "original_order"
    order.columns = ["orig_index", "datetime"]
    order.sort_values(by=["datetime", "orig_index"], inplace=True)
    order.reset_index(inplace=True, drop=True)

    # Returning results in Series format.
    result = pd.Series(data=order["orig_index"], index=order.index)

    return result

A lot of '01 values are incorrect.
- Example row 455 in the original data
- Row 463

In [74]:
dates = date_sorter(df)

# for i, date in dates.items():
#     print(i, date)

dates

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
30     162
31     154
32     402
33      95
34      73
35     108
36     156
37     332
38     182
39      82
40     351
41     278
42     214
43     155
44     223
45     473
46      49
47     317
48      11
49     319
50      40
51     418
52     165
53     370
54     382
55       3
56      50
57     363
58     219
59     465
60     237
61      23
62     342
63     204
64     258
65     315
66      27
67      93
68      17
69     303
70     488
71     283
72     395
73     309
74     419
75     123
76      19
77     117
78     232
79      72
80     189
81     318
82     369
83     493
84     239
85     148
86     105
87     336
88       6
89     200
90      81